In [0]:
dbutils.fs.mount(
source = "wasbs://smartmeterdata@myiotstrg.blob.core.windows.net",
mount_point = "/mnt/smartmeterdata",
extra_configs = {"fs.azure.account.key.myiotstrg.blob.core.windows.net": "raFPox2dC6XiEYpKH1h+KTp065IL4E7pu6kqu/n6zN3UDAQ3TffUoSC4xyof50p9g++yrub4sHvn+AStTNjL8g==" })




Out[23]: True

In [0]:
display(dbutils.fs.ls("/mnt/smartmeterdata"))

path,name,size,modificationTime
dbfs:/mnt/smartmeterdata/smartmeterdata/,smartmeterdata/,0,0


In [0]:
df =  spark.read.options(header='true',inferSchema='true').csv("dbfs:/mnt/smartmeterdata/smartmeterdata/*/*/*/*.csv",header=True)
print(df.dtypes)


[('id', 'string'), ('time', 'timestamp'), ('temp', 'int'), ('voltage', 'double'), ('EventProcessedUtcTime', 'timestamp'), ('PartitionId', 'int'), ('EventEnqueuedUtcTime', 'timestamp'), ('IoTHub', 'string')]


In [0]:
df.show(10)

+-------+--------------------+----+-------------------+---------------------+-----------+--------------------+------+
|     id|                time|temp|            voltage|EventProcessedUtcTime|PartitionId|EventEnqueuedUtcTime|IoTHub|
+-------+--------------------+----+-------------------+---------------------+-----------+--------------------+------+
|Device3|2023-03-24 02:28:...|  99| 0.5673613705958059| 2023-03-24 02:28:...|          1|2023-03-24 02:28:...|Record|
|Device0|2023-03-24 02:28:...|  94| 0.5732943126313828| 2023-03-24 02:28:...|          1|2023-03-24 02:28:...|Record|
|Device2|2023-03-24 02:28:...| 105|  0.965004266360311| 2023-03-24 02:28:...|          1|2023-03-24 02:28:...|Record|
|Device4|2023-03-24 02:28:...| 105|0.31017014645234225| 2023-03-24 02:28:...|          1|2023-03-24 02:28:...|Record|
|Device1|2023-03-24 02:28:...|  99| 0.7538585807419654| 2023-03-24 02:28:...|          1|2023-03-24 02:28:...|Record|
|Device3|2023-03-24 02:28:...| 104| 0.4320169802410607| 

In [0]:
df.write.mode("overwrite").saveAsTable("SmartMeters")

In [0]:
%sql
select id,count(*) as count,avg(temp) as AverageTemp from SmartMeters group by id order by id

id,count,AverageTemp
Device0,606,99.4042904290429
Device1,607,99.38879736408566
Device2,674,99.45845697329376
Device3,606,99.47854785478548
Device4,610,99.38032786885246


In [0]:
summary = spark.sql("select id,count(*) as count,avg(temp) as AverageTemp from SmartMeters group by id order by id")
summary.show()

summary.write.mode("overwrite").saveAsTable("DeviceSummary")

+-------+-----+-----------------+
|     id|count|      AverageTemp|
+-------+-----+-----------------+
|Device0|  606| 99.4042904290429|
|Device1|  607|99.38879736408566|
|Device2|  674|99.45845697329376|
|Device3|  606|99.47854785478548|
|Device4|  610|99.38032786885246|
+-------+-----+-----------------+



In [0]:
%sql
select * from DeviceSummary

id,count,AverageTemp
Device0,606,99.4042904290429
Device1,607,99.38879736408566
Device2,674,99.45845697329376
Device3,606,99.47854785478548
Device4,610,99.38032786885246


Output can only be rendered in Databricks